# How to be a Bioinformagician part 1
- Author: Olga Botvinnik
- Date: 2018-10-09

Abstract: Many computational problems have already been solved and yet hundreds of hours are lost to re-solving them. This series provides tips and tricks that solve common pain points in bioinformatics, using AWS, reading/writing CSVs, extracting data out of file names, and more.

## Prerequisities
1. Everything you do should be full screen. Use [divvy](http://mizage.com/divvy/) to make managing windows easier - ask IT for a license
1. [anaconda python](https://www.anaconda.com/download/#macos) - Install the 3.x (e.g. 3.7) version if you don't have it already
1. aws account - Ask Olga or James for one
1. awscli - on the command line, `pip install awscli` after you install Anaconda Python
1. aegea - on the command line, `conda install aegea` after you install Anaconda Python
1. nbconda - on the command line, `conda install nbconda` after you install Anaconda Python
1. reflow. Click this link: https://github.com/grailbio/reflow/releases/download/reflow0.6.8/reflow0.6.8.darwin.amd64 Then on the command line:
    ```
    cd ~/Downloads
    chmod ugo+x reflow0.6.8.darwin.amd64
    sudo mv reflow0.6.8.darwin.amd64 /usr/local/bin/reflow
    ```
    Now the command `reflow` should output a lot of stuff:
    
    ```
      reflow
    The reflow command helps users run Reflow programs, inspect their
    outputs, and query their statuses.

    The command comprises a set of subcommands; the list of supported
    commands can be obtained by running

        reflow -help

    ... (more stuff) ...
    ```
    Then configure reflow, following the [Confluence entry on Reflow (https://czbiohub.atlassian.net/wiki/spaces/DS/pages/838205454/reflow) instructions for configuration:
    ```
    AWS_SDK_LOAD_CONFIG=1 reflow setup-ec2
    AWS_SDK_LOAD_CONFIG=1 reflow setup-s3-repository czbiohub-reflow-quickstart-cache
    AWS_SDK_LOAD_CONFIG=1 reflow setup-dynamodb-assoc czbiohub-reflow-quickstart
    export AWS_REGION=us-west-2
    ```
1. Claim a folder within `s3://czbiohub-cupcakes/` with today's date and your username, e.g.:
    ```
    s3://czbiohub-cupcakes/2018-10-09/olgabot/
    ```


Highly recommended:
- If you haven't seen it already, follow https://github.com/czbiohub/codonboarding
- Especially 
    - install homebrew - it makes your life better for installing packages on mac
    ```
    /usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
    ```
    - install Oh My ZSH: https://ohmyz.sh/
    - Install exa: https://the.exa.website/ - MUCH easier to install if you installed homebrew.
    ```
    brew install exa
    ```

### Note: This notebook *can* be run locally 

But there's some weird version issues with conda packages on Mac and anyway it's cooler to launch from AWS so that's what we're doing

## Installation

### 1. Clone the cupcakes repo to `~/code` if you haven't already


```
mkdir ~/code
cd ~/code
git clone https://github.com/czbiohub/cupcakes
```

### 2. Create a "bioinformagician" environment

```
conda env create --name bioinformagician --file cupcakes/2018/olgas_bioinformagician_tricks/environment_no_versions_no_ng.yml
```

Then activate the environment

```
source activate bioinformagician

```

## Now we are ready to read the data!

Run each cell below by pressing Shift+Enter


1. Helpful jupter notebook keystrokes: 
    - Ctrl-M-A add cell above
    - Ctrl-M-B add cell below
    - Ctrl-M-d d delete cell
    - Ctrl-M-i interrupt
    - Ctrl-M-0 restart

The ones above are the shortcuts I use the most. Go to Help > Keyboard Shortcuts to see them all.

In [1]:
# Standard convention is to import python standard libraries first, then third-party libraries after that
# See list of standard libraries here: https://docs.python.org/3/library/
# Both import lists should be alphabetically sorted

# --- Python standard library --- #
# Easily grab filenames from a folder
import glob

# Amazing library that I use almost every day for:
# - chaining lists together into mega-lists
# - "multiplying" lists against each other to get the full product of combinations
import itertools

# Read/write javascript object notation (JSON) files
import json

# Perform path manipulations
import os

# --- Third-party (non-standard Python) libraries --- #
# python dataframes. very similar to R dataframes
import pandas as pd

# Make the number of characters allowed per column super big since our filenames are long
pd.options.display.max_colwidth = 500

### Read csv of lung cancer fastqs for which a kmer signature was calculated

In [2]:
compute_samples = pd.read_csv('lung-cancer/compute/samples.csv')
print(compute_samples.shape)
compute_samples.head()

(5054, 10)


,id,read1,read2,name,output,trim_low_abundance_kmers,dna,protein,ksizes,scaled
0,A10_B000419_S34,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000419_S34/A10_B000419_S34_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000419_S34/A10_B000419_S34_R2_001.fastq.gz,A10_B000419_S34,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,True,True,True,"21,27,33,51",1000
1,A10_B000420_S82,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000420_S82/A10_B000420_S82_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000420_S82/A10_B000420_S82_R2_001.fastq.gz,A10_B000420_S82,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,True,True,True,"21,27,33,51",1000
2,A10_B002073_S166,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002073_S166/A10_B002073_S166_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002073_S166/A10_B002073_S166_R2_001.fastq.gz,A10_B002073_S166,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002073_S166.signature,True,True,True,"21,27,33,51",1000
3,A10_B002078_S202,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002078_S202/A10_B002078_S202_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002078_S202/A10_B002078_S202_R2_001.fastq.gz,A10_B002078_S202,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002078_S202.signature,True,True,True,"21,27,33,51",1000
4,A10_B002095_S118,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002095_S118/A10_B002095_S118_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002095_S118/A10_B002095_S118_R2_001.fastq.gz,A10_B002095_S118,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002095_S118.signature,True,True,True,"21,27,33,51",1000


## Read the documentation of the `sourmash_search.rf` file to see what we need

In [3]:
! reflow doc reflow/sourmash_search.rf

Parameters

val signature string (required)
    S3 path to single signature file e.g.
    s3://olgabot-maca/facs/sourmash_compute_all/A1-B000610-3_56_F-1-1.sig
val database string (required)
    S3 full path to the sourmash database folder containing the database folder e.g.
    s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
    Note: this folder contains tabula-muris-k21-protein.sbt.json and a bunch of
    hidden files
val database_name string (required)
    Name of the database e.g.: tabula-muris-k21-protein
val output string (required)
    CSV file to write with search results e.g
    s3://olgabot-maca/facs/sourmash_search/A1-B000610-3_56_F-1-1_tabula-muris-k21-protein.csv
val ksize int = 21
    Size of kmer to use (can only use one for index)
val sequence_to_compare string = "dna"
    What to compare, could be either "protein" or "dna"
val ignore_abundance bool = false
    Whether or not to include the abundance of kmers in the comparis

We will be creating a *Reflow batch job* which will submit 100s of lung cancer signatures to be looked up in the tabula muris database.


For every argument, we'll need to create a column in a CSV that has that title, e.g. here we need the columns:

- signature
- database
- database_name
- output
- ksize
- sequence_to_compare
- ignore_abundance

## look at aws folder of tabula muris kmer signature databases and save aws output to file

In [ ]:
prefix = 's3://olgabot-maca/facs/sourmash_index_all'
txt = 'sourmash_databases.txt'

! aws s3 ls $prefix/ > $txt
! cat $txt

Show that we now have a file called `sourmash_databases.txt`

In [ ]:
ls -lha

Running each line one-by-one is left as an exercise to the reader :)

- To uncomment each cell, put your cursor in it, then:
    1. select-all with Command-A 
    2. uncomment with Command-/

In [ ]:
# databases = pd.read_table(txt, delim_whitespace=True, header=None, names=['is_prefix', 'database_name'])
# databases

In [ ]:
# databases['database_name'] = databases['database_name'].str.strip('/')
# databases

In [ ]:
# databases = databases.drop('is_prefix', axis=1)
# databases

In [ ]:
# databases['ksize'] = databases['database_name'].str.extract('k(\d+)').astype(int)
# databases

In [ ]:
# databases['sequence_to_compare'] = databases['database_name'].map(lambda x: x.split('-')[-1])
# databases

In [ ]:
# databases['database'] = databases['database_name'].map(lambda x: f"{prefix}/{x}/{x}/")
# databases

In [ ]:
# databases = databases.set_index('database_name')
# databases

### Load the table with pandas

In [ ]:
databases = pd.read_table(txt, delim_whitespace=True, header=None, names=['is_prefix', 'database_name'])
databases['database_name'] = databases['database_name'].str.strip('/')
databases = databases.drop('is_prefix', axis=1)
databases['ksize'] = databases['database_name'].str.extract('k(\d+)').astype(int)
databases['sequence_to_compare'] = databases['database_name'].map(lambda x: x.split('-')[-1])
databases['database'] = databases['database_name'].map(lambda x: f"{prefix}/{x}/{x}/")
databases = databases.set_index('database_name')
databases

### Only compare on protein databases

Since we're mapping human signatures onto a mouse database, we want to only compare on the protein signatures since protein sequences are more conserved than nucleotides

In [ ]:
protein_databases = databases.query('sequence_to_compare == "protein"')
protein_databases

Now comes the cool part!

## "Multiply" the samples x databases x ignore abundances to get every combination

We want to map each human lung sample onto ALL FOUR databases, PLUS we want to try both `ignore_abundance=True` and `ignore_abundance=False` 

Use `product` from Python's [itertools](https://docs.python.org/3/library/itertools.html) which is my favorite standard library module. (Though [collections](https://docs.python.org/3/library/collections.html) is a close second)

Remember that Python was designed as "batteries included" so if you're doing something like doing a ton of nested for loops, know that many people have done that in the past and have figured out better ways to do it.

In [ ]:
ignore_abundances = True, False

data = list(itertools.product(compute_samples['output'], ignore_abundances,
                              protein_databases.index))

samples = pd.DataFrame(data, columns=['signature', 'ignore_abundance', 'database_name'])
print(samples.shape)
samples

### Extract the `cell_id` from the signature filename

In [ ]:
samples['cell_id'] = samples['signature'].map(lambda x: os.path.basename(x).split('.')[0])
samples.head()

### Add an output location

This is the full path to where we'll be storing the output csv files from the search results

In [ ]:
# Change the following to the output location you chose! Should start with s3://czbiohub-cupcakes, 
# e.g. s3://czbiohub-cupcakes/2018-10-09/olgabot but use YOUR username! (not "olgabot!")
output_prefix = ''

samples['output'] = output_prefix + samples['database_name'] + '/' + samples['cell_id'] + '.csv'
samples['output'].head()

## Add the protein database information to the samples
We'll use the [`join()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) to combine the `protein_databases` table with our `samples` table, so we have the database `ksize`, `sequence_to_compare` and full URL.

[Here](https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/) is a nice blog post showing very clear examples of merge, join, and concatenate with pandas dataframes.

In [ ]:
samples_databases = samples.join(protein_databases, on='database_name')
print(samples_databases.shape)
samples_databases.head()

## Create a unique id for each row

Reflow will create a log file for every job, and if you have duplicate ids, then those logs will get overwritten, and it won't treat those jobs as unique. So you want to have UNIQUE ids for each row.

We use the [`apply()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) to run the same function on every *row* of the dataframe. If we hadn't specified `axis=1`, it would have tried to apply the function to every *column* (`axis=0`). I usually forget which one is which so I try the function on one side and if it doesn't work I know it's the opposite.

In [ ]:
samples_databases['id'] = samples_databases.apply(lambda x: 
                              '{cell_id}_ignore-abundance={ignore_abundance}_{database_name}'.format(**x), 
                              axis=1)
samples_databases.head()

## Subset to a few dozen cells
We don't need to see the result of ALL human cells, which is ~5,000. We can just look at the output of a few to get a feel for how well it is working. Below are the sample ids that I chose, and we'll subset using the [`.query()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.query.html) and access the python variable with `@chosen_ids`.

In [ ]:
chosen_ids = ['C14_B003528_S62',
 'D1_B003125_S25',
 'E19_B003570_S199',
 'F21_B000420_S213',
 'G10_B003586_S142',
 'G4_B003570_S232',
 'G9_B003511_S57',
 'H7_B003588_S211',
 'I22_B002095_S22',
 'I3_B003573_S63',
 'J11_B003573_S95',
 'J8_B003528_S224',
 'K7_B002073_S103',
 'L16_B003588_S16',
 'L5_B003588_S5',
 'M1_B000420_S61',
 'M23_B002097_S251',
 'N15_B000420_S99',
 'O3_B003573_S207',
 'P14_B000420_S146',
 'P2_B003125_S14']

samples_subset = samples_databases.query('cell_id in @chosen_ids')
print(samples_subset.shape)
samples_subset.head()

## Remove sample ID column and set

In [ ]:
samples_no_cell_id = samples_subset.drop(columns=['cell_id'])
samples_no_cell_id = samples_no_cell_id.set_index('id')
print(samples_no_cell_id.shape)
samples_no_cell_id.head()

## Create a folder to save reflow workflow to

What is our current directory?

In [ ]:
pwd

Make the folder

In [ ]:
folder = 'lung-cancer/search_protein_databases'
! mkdir $folder

Double-check that the folder exists

In [ ]:
ls -lha

### Write reflow batch `config.json` and `samples.csv` file

We'll run the program `sourmash_search.rf` which is in the `reflow` folder here. I recommend keeping your reflow scripts separate from their batch folders as you may use the same script across multiple folders.

In [ ]:
config = 	{
    # Since the folder we're writing to is relative to here as "lung-cancer/search_protein_databases"
    # but the reflow folder is "reflow/" then we need to go up two directories with "../.."
	"program": "../../reflow/sourmash_search.rf",
	"runs_file": "samples.csv"
	}

# Make sure the index (the ids!) are unique
assert samples_no_sample_id.index.is_unique

samples_no_sample_id.to_csv(f'{folder}/samples.csv', index=True)


with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)

### Look at the contents of the folder

In [ ]:
ls -lha $folder

#### Count the number of lines  in the folder to make sure it's the same as our input file

In [ ]:
! wc -l $folder/samples.csv

## Running reflow jobs


To run the reflow batch, go to the terminal and navigate to the `cupcakes/2018/olgas_bioinformagician_tricks/lung-cancer/search_protein_databases` directory. Once you're there, run this command:

```
reflow runbatch
```